In [1]:
import subprocess
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import requests

#pip install GitPython
import git

pd.set_option('display.max_rows',500) 

Git pull to update john hopkins dataset from github

In [2]:
g = git.cmd.Git(os.path.dirname('../data/raw/COVID-19/'))
out=g.pull()
print(out)

Already up to date.


In [3]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)


Its shows the number of cases confirmed on each day for provices/states(give longitude and latitude) of the countries 

WebScrapping

https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html 
The website shows the number of cases in each states of the germany. 
The values are obtained through webscrapping.

Inconsistent and unreliable,since webscrapping depends on the structure of web page. A minor update in the webpage can cause error.

In [4]:
page=requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [5]:
soup=BeautifulSoup(page.content,'html.parser')

In [6]:
html_table=soup.find('table')

In [7]:
all_rows=html_table.find_all('tr')

In [8]:
final_data_list=[]

In [9]:
for pos,rows in enumerate(all_rows):
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    final_data_list.append(col_list)

In [10]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                                      1:'cases',
                                                                      2:'changes',
                                                                      3:'cases_per_100k',
                                                                      4:'fatal',
                                                                      5:'comment'})

In [11]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,45.323,+69,1.269,"11,5",1.868
3,Bayern,62.401,+427,2.324,"17,8",2.647
4,Berlin,12.354,+85,486,"13,0",226
5,Branden­burg,4.016,+10,63,"2,5",169
6,Bremen,2.137,+2,64,"9,4",59


API calls

https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/ef4b445a53c1406892257fe63129a8ea_0/geoservice?geometry=-33.730%2C46.270%2C49.986%2C55.886

https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0/geoservice


https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json


In [12]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')
# data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')


In [13]:
json_object=json.loads(data.content)

In [14]:
type(json_object)

dict

In [15]:
json_object['features'][0]

{'attributes': {'OBJECTID_1': 1,
  'LAN_ew_AGS': '01',
  'LAN_ew_GEN': 'Schleswig-Holstein',
  'LAN_ew_BEZ': 'Land',
  'LAN_ew_EWZ': 2896712,
  'OBJECTID': 15,
  'Fallzahl': 4255,
  'Aktualisierung': 1600120800000,
  'AGS_TXT': '01',
  'GlobalID': 'fc5ba936-c95c-432c-8a33-9eb2f30b660f',
  'faelle_100000_EW': 146.890681572763,
  'Shape__Area': 45737314116.8701,
  'Shape__Length': 2881496.1295035,
  'Death': 161},
 'geometry': {'rings': [[[9.72694855217059, 53.5586543755165],
    [9.72953698118645, 53.5582212414827],
    [9.73031519620866, 53.5580193080416],
    [9.73010405438765, 53.5575809108903],
    [9.71926740492579, 53.5583267696142],
    [9.71627908277636, 53.5585171416657],
    [9.71743713604342, 53.5586307514866],
    [9.7185850167882, 53.5587521461929],
    [9.71954430073065, 53.5587011502349],
    [9.72069675929011, 53.5585657713295],
    [9.72172008862075, 53.558572225221],
    [9.72321985922525, 53.5587028673542],
    [9.72521088791592, 53.5587692928162],
    [9.726948552170

In [16]:
full_list=[]
for pos,each_dict in enumerate(json_object['features'][:]):
    full_list.append(each_dict['attributes'])
    

In [17]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4255,1600120800000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,146.890682,4.573731e+10,2.881496e+06,161
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6846,1600120800000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,371.826965,2.089396e+09,4.188002e+05,268
2,3,03,Niedersachsen,Land,7982448,9,18023,1600120800000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,225.782868,1.299836e+11,4.008988e+06,667
3,4,04,Bremen,Freie Hansestadt,682986,5,2137,1600120800000,04,4132268b-54de-4327-ac1e-760e915112f1,312.890747,1.119157e+09,3.357177e+05,59
4,5,05,Nordrhein-Westfalen,Land,17932651,10,62725,1600120800000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,349.780967,8.782936e+10,2.648673e+06,1834


In [18]:
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

API Access via Rest service 
NP

api.smartable.ai not available

In [19]:
# url_endpoint='https://api.smartable.ai/coronavirus/stats/NP'
# headers={
#     'Cache-Control':'no-cache',
#     'Subscription-Key':'631ee115ef754ed1a9668c1d33cd4168',
# }
# response=requests.get(url_endpoint,headers=headers)

In [20]:
# print(response)

In [21]:
# NP_dict=json.loads(response.content)
# with open('../data/raw/SMARTABLE/NP_data.txt','w') as outfile:
#     json.dump(NP_dict,outfile,indent=2)

In [22]:
# NP_dict